In [ ]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [ ]:
df   = pd.read_csv('dataset/disaster.csv')
text = df['text'][2]

In [ ]:
def prepare_json(text):
    feature_spec = {
        'text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, 'utf-8')]))
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result
    })

In [ ]:
def predict(text):
    response = requests.post(
        'http://localhost:8080/v1/models/disaster-tweets:predict',
        data = prepare_json(text)
    )

    predict = response.json()['predictions'][0][0]
    return predict

In [ ]:
predict = predict(text)

print(f'{text}\n')
print(f'Tweet Asli ({predict})' if predict > 0.6 else f'Tweet Palsu ({predict})')